In [1]:
import pandas as pd
from transformers import XLMRobertaModel, XLMRobertaTokenizer
import torch

tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base", pad_token = "<pad>")
model = XLMRobertaModel.from_pretrained("xlm-roberta-base")

C:\Users\germa\AppData\Local\Temp\ipykernel_8884\990675155.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
c:\Users\germa\anaconda3\envs\project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data = pd.read_csv("../../data/train/train.csv")
test_data = pd.read_csv("../../data/test/final_test_pairs.csv")

print(train_data.columns)

Index(['Unnamed: 0', 'pair_id', 'id1', 'id2', 'text1', 'text2', 'overall',
       'lang1', 'lang2'],
      dtype='object')


In [3]:
def select_tokens(n_head: int, n_tail: int, text: str):
    """
        Select n tokens from either head or tail of the text.

        Parameters:
            n_head: number of tokens from the head of the text
            n_tail: number of tokens from the tail of the text
            text: text to select tokens from
    """

    tokens = text.split()
    n_tokens = len(tokens)

    if n_tokens <= n_head + n_tail:
        return text

    head = tokens[:n_head]
    tail = tokens[-n_tail:]

    return " ".join(head + tail)

# # truncate the text by getting N_HEAD tokens from the head and N_TAIL tokens from the tail
# train_data["text1_truncated"] = train_data["text1"].apply(lambda x: select_tokens(N_HEAD, N_TAIL, x))
# train_data["text2_truncated"] = train_data["text2"].apply(lambda x: select_tokens(N_HEAD, N_TAIL, x))

# # concatenate the truncated texts with a separator token
# train_data["text_truncated"] = train_data["text1_truncated"] + " <sep> " + train_data["text2_truncated"]

# Get the embeddings for the train set

In [4]:
# def pad_embedding(embedding, target_length):
#     """
#     Pads the embedding tensor to the target length with zeros.

#     Parameters:
#         embedding: The embedding tensor of shape [1, sequence_length, embedding_size].
#         target_length: The target sequence length.
#     """
#     current_length = embedding.size(1)
#     padding_length = target_length - current_length
#     if padding_length > 0:
#         # padding is added as [1, padding_length, embedding_size]
#         padding = torch.zeros((1, padding_length, embedding.size(2)))
#         padded_embedding = torch.cat([embedding, padding], dim=1)
#         return padded_embedding
#     else:
#         return embedding

In [5]:
# train_embeddings = torch.empty((len(train_data), 512, 768))

# for i, text in enumerate(train_data["text_truncated"]):
#     inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
#     with torch.no_grad():
#         outputs = model(**inputs)
#         embeddings = outputs.last_hidden_state
#         embeddings = pad_embedding(embeddings, 512)
        
#     train_embeddings[i] = embeddings

# print(train_embeddings.shape)

In [6]:
def pad_embedding(text, target_length):
    text = torch.tensor(text)
    #add ones to the end of the text (1 is the padding token)
    text = torch.nn.functional.pad(text, (0, target_length - text.shape[1]), 'constant', 1)
    return text


In [7]:
def tokenize_and_shorten_sentence(text):
    
    tokenized_text = tokenizer(text, return_tensors="pt", padding=False, truncation=False)
    #padding = False and truncation = False to get the exact length of the text
    if tokenized_text["input_ids"].shape[1] > 256:
        tokenized_text["shorten_ids"] = torch.cat((tokenized_text["input_ids"][:, :200], tokenized_text["input_ids"][:, -56:]), dim=1)
    else:
        tokenized_text["shorten_ids"] = tokenizer(text, return_tensors="pt", padding="max_length", truncation=False, max_length=256)["input_ids"]
    return tokenized_text["shorten_ids"]

In [10]:
train_embeddings = torch.empty((len(train_data), 512, 768))

for i in range(len(train_data)):
    text1 = train_data["text1"][i]
    text2 = train_data["text2"][i]
    text1_tokenized = tokenize_and_shorten_sentence(text1)
    text2_tokenized = tokenize_and_shorten_sentence(text2)
    print(tokenizer.decode(text1_tokenized[0]))
    print(text1_tokenized.shape)

    

<s> MARTINSBURG, W.Va. — A suspected drunken driver was arrested in a New Year’s Day highway crash that killed another motorist in West Virginia, police said.The accident occurred early Wednesday along southbound Interstate 81 in Berkeley County, The Journal reported.Cody Wade Braithwaite, 32, of Winchester, Virginia, was charged with DUI with death, fleeing DUI, and being a prohibited person in possession of a firearm and a fugitive from justice, Berkeley County Sheriff Curtis Keller said.Keller said a deputy investigating calls about a reckless driver attempted to make a traffic stop, but the suspect’s vehicle sped up and hit a car, which then struck a guardrail. The driver of the car was pronounced dead at the scene. The driver’s name was not immediately released.The suspect hit a deputy’s cruiser before being stopped, Keller said.Braithwaite was being held without bond in the Eastern Regional Jail. It wasn’t immediately known whether Braithwaite has an attorney.___Information from:

KeyboardInterrupt: 

Hubert, I modified it so it now gets the first 200 subwords and the last 56 subwords of the sentence for each text.
If the sentence is shorter than 256 subwords, it gets padded with 1 (padding token id).

If you check train_data["text1"][10], that it needs padding, you will see that the pad token is added BUT after the end of sentence token </s>.
the model automatically does the padding after the eos_token, so we need to think if this is a problem or not